<font size="4"> The goal of this model is to be able to recognize which painter of 50 artists painted certain painting.<br>
Dataset used for training this model is taken from:<br>
 - https://www.kaggle.com/ikarus777/best-artworks-of-all-time?select=resized
</font>

In [ ]:
import tensorflow as tf
import pathlib
import os
import re
import cv2
import matplotlib.pyplot as plt
import numpy as np

<font size="4"> After downloading dataset, categories had to be pulled from filenames because dataset wasn't grouped in appropriate directories. Thanks to repeatable schema of filenames the best way to do it was to use regex' search and group methods. </font>

In [2]:
data_dir = "resized/"
categories = []
def get_categories():
    for name in os.listdir(data_dir):
        if not name.endswith('.jpg'):
            continue
        clean_name = re.search('(.*)_\d*.jpg', name)
        categories.append(clean_name.group(1))  
    return set(categories)

<font size="4"> In order to get best possible outcome of model's predictions, images in dataset had to be of the same size.<br>
    CV2 library is used below in order to get images' arrays which simply are just arrays of pixels that create certain image.<br>
    After resizing image it is assigned to its category.<br>
    Categories are represented with numbers because numeric values are understood best by model.</font>

In [18]:
IMG_SIZE = 200
pictures_set = []
categories = list(get_categories())
for name in os.listdir(data_dir):
    if not name.endswith('.jpg'):
        continue
    clean_name = re.search('(.*)_\d*.jpg', name)
    img_array = cv2.imread(os.path.join(data_dir, name))
    resized_img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    pictures_set.append([resized_img_array, categories.index(clean_name.group(1))])

<font size="4"> Data should be shuffled in order to avoid any possible bias.</font>

In [4]:
import random
random.shuffle(pictures_set)

<font size="4">Prepare X and y sets for training and testing model.<br>
Reshape features set so it represents image and RGB values of pixels.
</font>

In [5]:
X = []
y = []
for feature, category in pictures_set:
    X.append(feature)
    y.append(category)

X = np.asarray(X).reshape(-1,IMG_SIZE, IMG_SIZE, 3)

<font size="4"> Divide X and y sets for training and testing (validation) data. </font>

In [22]:
dataset_length = len(X)
X_train = X[:int(dataset_length*0.8)]
X_test = X[int(dataset_length*0.8):]
y_train = y[:int(dataset_length*0.8)]
y_test = y[int(dataset_length*0.8):]
print(
    len(X_train),
    len(X_test),
    len(y_train),
    len(y_test)
)

6684 1671 6684 1671


<font size="4"> Scale pixels' values so they will be standarized amongst all data.</font>

In [23]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

<font size="4"> Convert categories to binary matrix so they can be used in categorical crossentropy loss function. </font>

In [25]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

<font size="4"> Save feature set and category (class) set with pickle dump so it can be reused later on. </font>

In [6]:
import pickle

X_file = open('.X.pickle', 'wb')
pickle.dump(X, X_file)
X_file.close()

y_file = open('.y.pickle', 'wb')
pickle.dump(y, y_file)
y_file.close()

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

<font size="4"> I chose Sequential model blueprint which is appropriate for a stack of layers where each layer has exactly one input tensor and one output tensor. It is one of most commonly used blueprints.</font>

In [ ]:
model = Sequential()

<font size="4">Conv2D is convolutional layer which is a class of deep neural networks, usually it is applied to analyzing visual imagery.
Arguments of this layer:
- 32 - number of filters. Filter is what neural network uses to form a representation of the image,
- (3, 3) - size of filter,
- input_shape - shape of input that will go to convolutional layer (and progress further through layers),
- padding - 'same' argument defines that images will not be resized.</font>

In [ ]:
model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))

<font size="4">
    <ul>
 <li> Activation - Activation function / layer. Values that represent certain image are forwarded to activation layer which reduces linearity of this data.</li>
 <li> Dropout - Dropout layer is used for preventing overfitting of a model. It drops random connections between layers.</li>
 <li> BatchNormalization - It normalizes input that goes into next layer in order to preserve distribution.</li>
 <li> MaxPooling2D - Pooling function is a function that takes information about image and compress it. It makes network more efficient at recognizing images based on the relevant features. Max pooling function takes, as name suggests, maximum values of pixels in a filter. It drops a lot of useless information.</li>
 <li> Flatten - Final layers of convolutional neural network have to be in form of vector. This function makes them as such.</li>
 <li> Dense -  Dense function's aim is to analyze the input features and combine them into different attributes that will help in classification. These layers are collections of neurons that represent different parts of the image.</li>
    </ul>

After specyfing those layers i compile model with 'adam' optimizer which is algorithm that reduces loss by tuning weights appropriately.
</font>

In [ ]:
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(class_num))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

<font size="4">After compilation of model it can be trained on prepared sets. Arguments:
 - epochs - number of passes of the entire training dataset,
 - batch_size - number of training examples utilized in one iteration.</font>

In [31]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=32)

Epoch 1/2
209/209 [==============================] - 368s 2s/step - loss: 18.6956 - accuracy: 0.1457 - val_loss: 258.9274 - val_accuracy: 0.0413
Epoch 2/2
209/209 [==============================] - 379s 2s/step - loss: 15.0101 - accuracy: 0.3475 - val_loss: 224.5051 - val_accuracy: 0.0048


<font size="4"> I've only used two epochs because training a model is a very demanding process for GPU and it can take quite a long time. Still, after only two epochs we get almost 50% of accuracy in recognizing paintings from 50 categories.</font>

In [34]:
scores[1]*100

0.4787552170455456